# SciPy Integration

numerical quadrature, multiple integration, ODE solving, and cumulative integration with practical examples.

In [1]:
import numpy as np
from scipy import integrate

np.set_printoptions(precision=4, suppress=True)

## 1. Single-Variable Numerical Integration
Single-variable integration computes totals like fuel, cost, or expected loss over continuous ranges.

In [2]:
def fuel_rate(distance_km):
    return 0.055 + 0.00003 * distance_km + 0.002 * np.sin(distance_km / 40)

trip_distance = 420
total_fuel, err_fuel = integrate.quad(fuel_rate, 0, trip_distance)

print("Example 1: Fleet fuel planning")
print(f"Total fuel over {trip_distance} km: {total_fuel:.3f} liters")
print(f"Estimated error: {err_fuel:.3e}")

mu = 3000
sigma = 700

def expected_shortfall(x):
    density = 1 / (sigma * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)
    return max(0, 2500 - x) * density

loss, err_loss = integrate.quad(expected_shortfall, 0, 6000)

print()
print("Example 2: Expected shortfall")
print(f"Expected shortfall: {loss:.2f} units")
print(f"Estimated error: {err_loss:.3e}")

Example 1: Fleet fuel planning
Total fuel over 420 km: 25.864 liters
Estimated error: 1.705e-12

Example 2: Expected shortfall
Expected shortfall: 97.59 units
Estimated error: 7.252e-08


## 2. Multiple Integration
Multiple integrals compute probabilities, volumes, and aggregate quantities over 2D and 3D domains.

In [3]:
def joint_pdf(y, x):
    if 0 <= x <= 1 and 0 <= y <= 1:
        return 4 * x * y
    return 0

prob, _ = integrate.dblquad(lambda y, x: joint_pdf(y, x), 0, 1, lambda x: max(0, 1.2 - x), lambda x: 1)

print("Example 1: Probability over region")
print(f"P(X + Y > 1.2): {prob:.4f}")

def terrain(y, x):
    z = 25 - 0.35 * x ** 2 - 0.2 * y ** 2
    return max(z, 0)

volume, _ = integrate.dblquad(lambda y, x: terrain(y, x), -6, 6, lambda x: -5, lambda x: 5)

print()
print("Example 2: Earthwork volume")
print(f"Volume under terrain: {volume:.2f}")

Example 1: Probability over region
P(X + Y > 1.2): 0.6656

Example 2: Earthwork volume
Volume under terrain: 2296.00


## 3. Solving ODEs with solve_ivp
ODE solvers model dynamic systems such as thermal processes and epidemics.

In [4]:
def cooling_model(t, temp):
    ambient = 24
    return -0.12 * (temp - ambient)

cool = integrate.solve_ivp(cooling_model, t_span=(0, 40), y0=[92], t_eval=np.linspace(0, 40, 41))

print("Example 1: Cooling process")
print(f"Temperature at 10 min: {cool.y[0][10]:.2f}")
print(f"Temperature at 30 min: {cool.y[0][30]:.2f}")

def sir_model(t, y):
    s, i, r = y
    beta = 0.35
    gamma = 0.11
    return [-beta * s * i, beta * s * i - gamma * i, gamma * i]

sir = integrate.solve_ivp(sir_model, t_span=(0, 120), y0=[0.99, 0.01, 0.0], t_eval=np.linspace(0, 120, 121))
peak_i = np.argmax(sir.y[1])

print()
print("Example 2: SIR epidemic dynamics")
print(f"Peak infected share: {sir.y[1][peak_i]:.4f}")
print(f"Day of peak: {sir.t[peak_i]:.1f}")

Example 1: Cooling process
Temperature at 10 min: 44.50
Temperature at 30 min: 25.86

Example 2: SIR epidemic dynamics
Peak infected share: 0.3254
Day of peak: 23.0


## 4. Method Choice and Cumulative Integration
Method choice affects efficiency for stiff systems, and cumulative integrals track running totals from sampled data.

In [5]:
def stiff_decay(t, y):
    return -18 * y

rk = integrate.solve_ivp(stiff_decay, (0, 2), [1.0], method='RK45')
radau = integrate.solve_ivp(stiff_decay, (0, 2), [1.0], method='Radau')

print("Example 1: Stiff decay solver comparison")
print(f"RK45 evaluations: {rk.nfev}")
print(f"Radau evaluations: {radau.nfev}")
print(f"Final value RK45: {rk.y[0, -1]:.6f}")
print(f"Final value Radau: {radau.y[0, -1]:.6f}")

time = np.linspace(0, 24, 97)
power_kw = 2.8 + 1.3 * np.sin(2 * np.pi * time / 24 - 0.7) + 0.4 * np.sin(2 * np.pi * time / 8)
energy = integrate.cumulative_trapezoid(power_kw, time, initial=0)

print()
print("Example 2: Energy accumulation")
print(f"Energy after 8 hours: {energy[np.argmin(np.abs(time - 8))]:.3f} kWh")
print(f"Energy after 24 hours: {energy[-1]:.3f} kWh")

Example 1: Stiff decay solver comparison
RK45 evaluations: 134
Radau evaluations: 156
Final value RK45: 0.000000
Final value Radau: 0.000000

Example 2: Energy accumulation
Energy after 8 hours: 25.325 kWh
Energy after 24 hours: 67.200 kWh
